In [1]:
import os
import copy
import numpy as np
import torch
import einops
import pdb
import ipdb
from diffuser.utils.arrays import batch_to_device, to_np, to_device, apply_dict
from IPython.display import clear_output
import time

pybullet build time: May 20 2022 19:44:17


In [2]:
import diffuser.utils as utils

In [3]:
diffusion = utils.load_diffusion('../logs', 
                                 'maze2d-large-v1', 
                                 'diffusion/H384_T256_20231031-1406', 
                                 epoch='latest')
clear_output()

In [4]:
cond = {0: [1, 1, 0, 0], 383: [7, 9, 0, 0]}

cond = utils.apply_dict(
    diffusion.dataset.normalizer.normalize,
    cond,
    'observations'
)
cond = utils.to_torch(cond, dtype=torch.float32, device='cuda:0')
cond = utils.apply_dict(
    einops.repeat,
    cond,
    'd -> repeat d', repeat=1,
)

obst_position = [2.3, 2, 0, 0]
rad = 1

tmp1 = [0, 0, 0, 0]
tmp2 = [rad, rad, 0, 0]
normed_obst = diffusion.dataset.normalizer.normalize(obst_position, 'observations')
tmp1 = diffusion.dataset.normalizer.normalize(tmp1, 'observations')
tmp2 = diffusion.dataset.normalizer.normalize(tmp2, 'observations')

normed_rad = [(tmp2 - tmp1)[0], (tmp2 - tmp1)[1]]

dtau = 1
start = time.time()
samples = diffusion.ema.cbf_sample_vec(cond, cx=normed_obst[0], cy=normed_obst[1], radius=normed_rad, dtau=dtau)
end = time.time()
print(end-start)
samples = to_np(samples)
normed_observations = samples[:, :, 2:]
normed_conditions = to_np(cond[0])[:,None]
normed_observations = np.concatenate([
    np.repeat(normed_conditions, 1, axis=0),
    normed_observations
    ], axis=1)
observations = diffusion.dataset.normalizer.unnormalize(normed_observations, 'observations')
images = diffusion.renderer.composite('tmp.png', observations, ncol=1,
                                      obst_pos=np.array(obst_position[:2]), obst_rad=rad)

> /home/kazuki/diffuser/diffuser/models/diffusion.py(138)p_sample()
    137         ipdb.set_trace()
--> 138         nonzero_mask = (1 - (t == 0).float()).reshape(b, *((1,) * (len(x.shape) - 1)))
    139         return model_mean + nonzero_mask * (0.5 * model_log_variance).exp() * noise



ipdb>  model_log_variance.shape


torch.Size([1, 1, 1])


ipdb>  model_mean


tensor([[[ 2.2158e-02,  1.0856e-03, -3.0380e-02, -3.1925e-02, -1.8810e-04,
           7.1000e-05],
         [-6.5287e-03,  2.7106e-02, -1.0700e-02,  1.1587e-02, -3.0737e-02,
          -1.3338e-04],
         [ 2.2998e-03, -1.1592e-02,  3.1000e-02, -3.2945e-02, -1.4529e-02,
           2.5344e-02],
         ...,
         [ 1.7261e-02,  1.6202e-03,  5.3694e-04, -1.8089e-02,  5.8218e-02,
          -2.7431e-02],
         [ 1.7851e-02,  9.1052e-03,  2.1851e-02,  1.7245e-02,  7.7600e-05,
          -2.3188e-02],
         [ 2.3901e-02, -3.3216e-02,  3.4349e-02,  2.7491e-02, -4.2888e-04,
          -5.2352e-05]]], device='cuda:0')


ipdb>  model_mean.shape


torch.Size([1, 384, 6])


ipdb>  q
